In [4]:
import operator
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
ani = pd.read_csv('C:/Users/kehah/Desktop/anime rec/anime rec/anime.csv')
ratings = pd.read_csv('C:/Users/kehah/Desktop/anime rec/anime rec/rating.csv')

In [6]:
#결측 행 제거
ani.dropna(axis=0, inplace=True)
ani.shape

(12017, 7)

In [7]:
ratings['rating'].value_counts()

 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: rating, dtype: int64

In [8]:
ratings.replace(-1,0, inplace=True)
idx = ratings[ratings['rating'] == -1].index
ratings.drop(idx, inplace=True)


ratings.head(10)

,user_id,anime_id,rating
0,1,20,0
1,1,24,0
2,1,79,0
3,1,226,0
4,1,241,0
5,1,355,0
6,1,356,0
7,1,442,0
8,1,487,0
9,1,846,0


In [9]:
ratings_per_anime = ratings.groupby('anime_id')['rating'].count()

# 너무 적게 평가된 콘텐츠 제거
ratings_per_anime_df = pd.DataFrame(ratings_per_anime)

# 평점 개수가 100개 이하 콘텐츠 제거
filtered_ratings_per_anime_df = ratings_per_anime_df[ratings_per_anime_df.rating >= 100]
print('평점 갯수 100 개 이상 콘텐츠 :',filtered_ratings_per_anime_df.shape)
popular_anime = filtered_ratings_per_anime_df.index.tolist()

filtered_ratings_per_anime_df
#평점이 100개이상 평가된 애니 컨텐츠만 추출하고 header 제거한 파일 생성 
#filtered_ratings_per_anime_df.to_csv('C:/Users/kehah/Desktop/anime rec/anime rec/rating_noh.csv', index=False, header=False)

평점 갯수 100 개 이상 콘텐츠 : (4591, 1)


,rating
anime_id,
1,15509
5,6927
6,11077
7,2629
8,413
...,...
33558,866
33569,176
33964,248


In [10]:
rating_per_user = ratings.groupby('user_id')['rating'].count()

# 너무 적게 평가한 유저 제거
rating_per_user_df = pd.DataFrame(rating_per_user)

# 평점 데이터 200개 이하인 유저의 경우 제거
filtered_rating_per_user_df = rating_per_user_df[rating_per_user_df.rating >= 200]
prolific_users = filtered_rating_per_user_df.index.tolist()

print('평점 200개 이상 남긴 유저:',filtered_rating_per_user_df.shape)

평점 200개 이상 남긴 유저: (11278, 1)


In [11]:
filtered_ratings = ratings[ratings.anime_id.isin(popular_anime)]
filtered_ratings = ratings[ratings.user_id.isin(prolific_users)]

filtered_ratings
filtered_ratings.to_csv('C:/Users/kehah/Desktop/anime rec/anime rec/filtered_rating_noh.csv', index=False, header=False)

In [12]:
#필요한 모듈 불러오기
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [13]:
import pandas as pd

ratings_1 = pd.read_csv('C:/Users/kehah/Desktop/anime rec/anime rec/filtered_rating_noh.csv')

#ratings_noh.csv : ratings에서 인덱스, 헤더 제거한 파일 생성


In [14]:
ratings_1.head(3)

,5,6,8
0,5,15,6
1,5,17,6
2,5,18,6


In [39]:
ratings.isna().sum()

5    0
6    0
8    0
dtype: int64

In [41]:
ratings['rating'].value_counts()

8     1646019
0     1476496
7     1375287
9     1254096
10     955715
6      637775
5      282806
4      104291
3       41453
2       23150
1       16649
Name: rating, dtype: int64

In [17]:
from surprise import Reader

reader = Reader(line_format='user item rating', sep=',', rating_scale=(1,10))

#Dataset.load_from_file API를 이용해 판다스의 데이터프레임에서 파라미터는 데이터프레임
#그리고 사용자아이디, 아이템아이디, 평점순으로 칼럼이 정해져 있어야함.
data=Dataset.load_from_file('C:/Users/kehah/Desktop/anime rec/anime rec/filtered_rating_noh.csv', reader=reader)
    #ratings_noh.csv : ratings에서 인덱스, 헤더 제거한 파일 생성


In [18]:
# train:test 8:2(80:20)로 분리
trainset, testset = train_test_split(data, test_size=.20, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.8069


1.8068915330382198

### 교차검증, 하이퍼 파라미터 튜닝

In [88]:
#from surprise.model_selection import cross_validate

ratings = pd.read_csv('C:/Users/kehah/Desktop/anime rec/anime rec/rating.csv')

reader = Reader(rating_scale = (1, 10.0))
data = Dataset.load_from_df(ratings[['user_id', 'anime_id','rating']], reader)

#algo=SVD(random_state=0)
#cross_validate(algo, data, measures =['RMSE','MAE'], cv=5, verbose=True)

In [20]:
from surprise.model_selection import GridSearchCV

#n_epochs 기본값 20, n_factors 기본값 100
param_grid = {'n_epochs':[10, 20, 30], 'n_factors':[50,100,150]}

#cv=3 성능평가 rmse, mae
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.7896482330992745
{'n_epochs': 10, 'n_factors': 50}


In [26]:
from surprise.model_selection import GridSearchCV

#n_epochs 기본값 20, n_factors 기본값 100
param_grid = {'n_epochs':[1,5,10], 'n_factors':[10,30,50]}

#cv=3 성능평가 rmse, mae
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.782577287443725
{'n_epochs': 10, 'n_factors': 30}


In [28]:
from surprise.model_selection import GridSearchCV

#n_epochs 기본값 20, n_factors 기본값 100
param_grid = {'n_epochs':[7,10,13], 'n_factors':[25,30,35]}

#cv=3 성능평가 rmse, mae
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.7822548529323257
{'n_epochs': 7, 'n_factors': 35}


In [25]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating', sep= ',',rating_scale=(1,10))

data_folds = DatasetAutoFolds(ratings_file='C:/Users/kehah/Desktop/anime rec/anime rec/filtered_rating_noh.csv', reader=reader)

#전체 데이터 셋을 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

In [30]:
algo = SVD(n_epochs=7, n_factors=35, random_state=0)
algo.fit(trainset)

In [117]:
#애니 데이터 불러오기
anime = pd.read_csv('C:/Users/kehah/Desktop/anime rec/anime rec/anime.csv')

animeIds = ratings[ratings['user_id']==226]['anime_id']

if animeIds[animeIds==32281].count() == 0:
    print('해당 user 는 animeid  평점 없음')
if animeIds[animeIds==32281].count() != 0:
    print('해당 유저는 해당 애니 본 적 있음')

print(anime[anime['anime_id']==32281])

해당 user 는 animeid  평점 없음
   anime_id            name                                 genre   type  \
0     32281  Kimi no Na wa.  Drama, Romance, School, Supernatural  Movie   

  episodes  rating  members  
0        1    9.37   200630  


In [91]:
type(animeIds)

pandas.core.series.Series

In [118]:
print(anime[anime['anime_id']==32281])

   anime_id            name                                 genre   type  \
0     32281  Kimi no Na wa.  Drama, Romance, School, Supernatural  Movie   

  episodes  rating  members  
0        1    9.37   200630  


In [119]:
uid=str(226)
iid = str(32281)

pred = algo.predict(uid, iid, verbose=True)

user: 226        item: 32281      r_ui = None   est = 9.18   {'was_impossible': False}


In [120]:
def unseen_ani_sur(ratings, anime, user_id):
    #사용자가 평점매긴 모든 영화를 리스트로 변환
    seen_ani_list = ratings[ratings['user_id']==user_id]['anime_id'].tolist()
    
    #모든영화의 movieId를 리스트로 생성
    total_ani_list = anime['anime_id'].tolist()
    
    #전체 영화 리스트에서 사용자가 본 영화가 아닌(안 본) 영화만(movieId)을 리스트로 생성 
    unseen_ani_list = [anime for anime in total_ani_list if anime not in seen_ani_list]
    
    print('the number of already seen animes:',len(seen_ani_list),   
          'the number of unseen animes:',len(unseen_ani_list),   
          'total number of animes:',len(total_ani_list))   
    return unseen_ani_list
um = unseen_ani_sur(ratings, anime, 226)

the number of already seen animes: 742 the number of unseen animes: 11552 total number of animes: 12294


In [121]:
def recomm_ani_sur(algo, user_id, unseen_ani_list, top_n=10):
    
    predictions = [algo.predict(str(user_id), str(anime_id)) for anime_id in unseen_ani_list]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key = sortkey_est, reverse=True)
    
    top_predictions = predictions[:top_n]
    
    top_ani_ids = [int(pred.iid) for pred in top_predictions]
    
    top_ani_rating = [pred.est for pred in top_predictions]
    
    top_ani_names = anime[anime.anime_id.isin(top_ani_ids)]['name']
    
    top_ani_preds = [ (id, name, rating) for id, name, rating in zip(top_ani_ids, top_ani_names, top_ani_rating)]
    
    return top_ani_preds

um = unseen_ani_sur(ratings, anime, 226)
top_anime_preds = recomm_ani_sur(algo, 226, um, top_n=10)

print('######')
for top_anime in top_anime_preds:
    print(top_anime[1], ":", top_anime[2])

the number of already seen animes: 742 the number of unseen animes: 11552 total number of animes: 12294
######
Kimi no Na wa. : 9.176885003574514
Gintama° : 9.046640073380466
Shokugeki no Souma : 9.004363317664131
Diamond no Ace: Second Season : 8.945141836900731
Junjou Romantica 3 : 8.86972432189395
High☆Speed!: Free! Starting Days : 8.859035570699477
Hybrid Child : 8.814704451915933
Noblesse: Awakening : 8.770685111865546
Code Geass: Boukoku no Akito 3 - Kagayaku Mono Ten yori Otsu : 8.768341610618366
Gangsta. : 8.757700669958963
